# Author Identification with Natural Language Processing
### - 自然言語処理を用いた「著者判定」-  2019年1月15日


![title](https://images.unsplash.com/photo-1524995997946-a1c2e315a42f?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

【ゴール】  
・著者判定

【データセット】  
・Spooky

【カバー内容】  
・TF-IDF  
・ロジスティック回帰  
・ナイーブベイズ  
・サポートベクターマシン  
・アンサンブル学習  
・XG-Boost（勾配降下法）  
・Grid Search  
・Word Vectors  
・LSTM  
・GRU  


In [18]:
import pandas as pd
import numpy as np
# import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.corpus import stopwords

"Spooky"のデータセットをロード

In [21]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') 

データの概要を確認

In [22]:
print("The size of author in train dataset : {}".format(train.shape))
print("The size of author in test dataset : {}".format(test.shape))
print("The unique number of author in dataset : {}".format(train["author"].nunique()))
train.head()

The size of author in train dataset : (19579, 3)
The size of author in test dataset : (8392, 2)
The unique number of author in dataset : 3


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


多クラス分類における対数損失を評価指標としてモデルを評価する

In [23]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

scikit-learnのラベルエンコーダーを使ってデータ型を変換

In [24]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

学習データとテストデータに分割します

In [25]:
xtrain, xtest, ytrain, ytest = train_test_split(
    train.text.values, y, 
    stratify=y, 
    random_state=42, 
    test_size=0.1, 
    shuffle=True)

In [26]:
print(xtrain.shape)
print(xtest.shape)

(17621,)
(1958,)


## 1. シンプルな分類器

### 1-1. TF-IDF (Term Frequency - Inverse Document Frequency)  
による重要単語抽出を用いた著者推定を行う。
予測モデルは、ロジスティック回帰による多クラス分類（３クラス）を用います。  

【手順】　　  
・TF-IDFで各著者の文章の特徴を最も表す単語を抽出（ソート）する関数を作成    
・学習データとテストデータの説明変数に上記の関数を噛ませる  
・ロジスティック回帰に学習データを噛ませて予測モデルを作成   
・モデルの評価指標として "multi-class logarithmic loss" を用いる  

TfidfVectorizerを使うと、文章を特徴づける単語を探すことができる。  
ロジックは、文章内に出現する単語の「出現頻度」と「希少性」を掛け合わせた値Tfidfを算出するアルゴリズム。  

In [52]:
# 出現回数が３回以上の単語を対象に、説明変数を生成する
tfidf = TfidfVectorizer(min_df=3, max_features=None, 
            strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

# 文章内の単語のTfidf値を取得
tfidf.fit(list(xtrain) + list(xtest))
xtrain_tfidf = tfidf.transform(xtrain)
xtest_tfidf = tfidf.transform(xtest)

ロジスティック回帰では、活性化関数をロジット関数 logit(x) としているため、デフォルトでは二値分類問題にしか適用できない。  
多クラス分類問題に拡張するとき、多クラスのラベルを表現するための式を導入し、活性化関数を変更する必要がある。  
機械学習の分野において、多クラスを表現するには one-hot 表現が一般的に使われている。  
この時、活性化関数はソフトマックス関数にする。

![title](https://www.infiniteloop.co.jp/blog/wp-content/uploads/2017/12/multi-to-binary-classification-640x165.png)

In [28]:
# ロジスティック回帰をTfidf上で行う
lr = LogisticRegression(C=1.0)
lr.fit(xtrain_tfidf, ytrain)
predictions = lr.predict_proba(xtest_tfidf)

# multi-class logarithmic loss は0に近いほどいい
print ("対数損失 multi-class logarithmic loss : %0.3f " % multiclass_logloss(ytest, predictions))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


対数損失 multi-class logarithmic loss : 0.617 


【結論】  
・TF-IDFのみでは、あまり精度が出なかった  

### 1-2. CountVectorizer による特徴量生成
  
TF-IDFでは各単語の出現回数に対して、文章全体における希少性も考慮した。  
今度はCountVectorizerを用いて純粋に単語の出現回数を基準に各著者ごとの特徴を作り出す。

In [29]:
cv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3))

cv.fit(list(xtrain) + list(xtest))
xtrain_cv = cv.transform(xtrain)
xtest_cv = cv.transform(xtest)

In [30]:
lr = LogisticRegression(C=1.0)
lr.fit(xtrain_cv, ytrain)
predictions = lr.predict_proba(xtest_cv)

print("対数損失 multi-class logarithmic loss : %0.3f " % multiclass_logloss(ytest, predictions))

対数損失 multi-class logarithmic loss : 0.457 


### 1-3. Naive Bayes - 単純ベイズ分類器 -

![title](https://camo.qiitausercontent.com/77351cc8720301944b086011836dca40ef0f65ca/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3136313639312f30323863323264392d346339612d653065322d393565622d3063386361343931663936372e706e67)

In [31]:
# まずはTfidfによって生成した特徴を説明変数とする
nbc = MultinomialNB()
nbc.fit(xtrain_tfidf, ytrain)
predictions = nbc.predict_proba(xtest_tfidf)
print ("対数損失 multi-class logarithmic loss : %0.3f " % multiclass_logloss(ytest, predictions))

対数損失 multi-class logarithmic loss : 0.574 


In [32]:
# 次はCountVectorizerによって生成した特徴を説明変数とする
nbc = MultinomialNB()
nbc.fit(xtrain_cv, ytrain)
predictions = nbc.predict_proba(xtest_cv)
print ("対数損失 multi-class logarithmic loss : %0.3f " % multiclass_logloss(ytest, predictions))

対数損失 multi-class logarithmic loss : 0.755 


### 1-4. Singular Value Decomposition - 特異値分解 -



In [33]:
# 特異値分解によってベクトル化した単語の次元を重要度の低い順から削除する
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfidf)
xtrain_svd = svd.transform(xtrain_tfidf)
xtest_svd = svd.transform(xtest_tfidf)

#### 特徴量の正規化と標準化


特徴量の尺度について。特徴量の尺度を揃えなさい、揃え方には正規化と標準化がある。  
多くの機械学習アルゴリズムでは標準化、つまり標準偏差で割ることが実用的とのこと。  
Scikit-learnでは、StandardScaler（標準化）、 MinMaxScaler（正規化）である。  
  
・fit パラメータ（平均や標準偏差 etc）計算  
・transform パラメータをもとにデータ変換  
・fit_transform パラメータ計算とデータ変換をまとめて実行  

![title](https://i.stack.imgur.com/rKSuk.png)

In [34]:
# 次元削減後の次元を標準化する
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
scl_xtrain_svd = scl.transform(xtrain_svd)
scl_xtest_svd = scl.transform(xtest_svd)

ここではサポートベクターマシン分類を予測モデルとして適応する

In [36]:
svc = SVC(C=1.0, probability=True)
svc.fit(scl_xtrain_svd, ytrain)
predictions = svc.predict_proba(scl_xtest_svd)

print ("対数損失 multi-class logarithmic loss : %0.3f " % multiclass_logloss(ytest, predictions))

対数損失 multi-class logarithmic loss : 0.700 


### XGBOOST

## Grid Search ハイパーパラメータの最適化  
   
正則化やガンマ分布など過学習や未学習の調整を自動で行う  

refference : http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/

【手順】    
・スコアラーの作成　　　  
・SVDによる次元削減      
・次元の標準化      
・ロジスティック回帰    
    
ここでは、scikit-learnの"make_scorer"関数を使う。

In [37]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better = False, needs_proba = True)

特徴量選択、前処理（スケール調整）、パラメータ設定、機械学習アルゴリズムの選択…という一連のプロセスを一つのEstimatorとして定義できると、API化が容易になったり、あるいはGridSearchによる自動化ができる。    
    
Pipelineを用いると、以下の点が付け加えられる。      
    
・特徴量選択や前処理のパラメータ設定の自動化   
・機械学習アルゴリズムや前処理メソッドの自動選択  

In [44]:
svd = TruncatedSVD()
scl = preprocessing.StandardScaler()
lr = LogisticRegression()
clf = pipeline.Pipeline([('svd', svd), ('scl', scl), ('lr', lr)])

「正則化」(regularization / penalized regression)のとは、モデル最適化のための誤差関数ED(w)に対して、ペナルティ項E(w)を加える。
ED(w)+λE(w)を最小化するように最適化問題を解く問題に置き換えることで、「ほどよく」過学習を避けつつ穏当なモデルに落ち付かせることを目的としている。  
   
・L1正則化回帰はLasso回帰   
・L2正則化回帰はRidge回帰   
・L1 / L2正則化項はElastic net正則化   

In [54]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [56]:
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# フィッティング
model.fit(xtrain_tfidf, ytrain)
print("ベストスコア: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:  2.3min remaining:   27.9s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.6min finished
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


ベストスコア: -0.691
Best parameters set:
	lr__C: 1.0
	lr__penalty: 'l2'
	svd__n_components: 180


単純ベイズを分類器として使う

In [58]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfidf, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.6s finished


Best score: -0.451
Best parameters set:
	nb__alpha: 0.1


## Word Vectors - 単語のベクトル表現 -

## ディープラーニング TensorFlow

## アンサンブル学習

In [ ]:
https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle
    